In [3]:
import pandas as pd
import os
import json
# __set_dir__()
os.path.abspath(os.path.join('data'))

'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\wwl_ds_projects\\localization_module\\data'

# Data Cleaning

In [4]:
path_list = []
folder_path = os.path.abspath(os.path.join('data','raw'))
for item in os.listdir(folder_path):
    if item.lower().endswith('.csv'):
        path_list.append(os.path.join(folder_path, item))
    
path_list

['c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\wwl_ds_projects\\localization_module\\data\\raw\\localization2_container_pose_2024-05-19_234444.csv',
 'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\wwl_ds_projects\\localization_module\\data\\raw\\localization2_container_pose_2024-05-20_052736.csv',
 'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\wwl_ds_projects\\localization_module\\data\\raw\\localization2_container_pose_2024-05-20_114311.csv',
 'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\wwl_ds_projects\\localization_module\\data\\raw\\localization2_correct_pose_by_wall_2024-05-18_073904.csv',
 'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\wwl_ds_projects\\localization_module\\data\\raw\\localization2_correct_pose_by_wall_2024-05-19_005440.csv',
 'c:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\wwl_ds_projects\\localization_module\\data\\raw\\localization2_correct_pose_by_wall_2024-05-20_052736.csv',
 'c:\\Users\\westwell\

In [5]:
df_list = []
name_list = [
    'localization2_gnss_pose',
    'localization2_localization_odom',
    'localization2_correct_pose_by_wall',
    'localization2_container_pose',
    'localization2_qc_match_pose',
    'localization2_optimized_pose',
    'localization2_magnetic_nail',
    'localization2_correct_pose_by_lane',
]

name = name_list[5]

def lmd_stamp(timestamp):
    '''
    Timestamp alignment, all ending in even numbers.
    '''
    if round(timestamp, 1) % 2 != 0: return round(timestamp, 1)-0.1
    else: return round(timestamp,1)

for p in path_list:
    if name in p:
        print(p)
        df_list.append(pd.read_csv(p))

df = pd.concat(df_list, axis=0)
display(df.columns)
display(df['stamp'].iloc[0])
display((round(df['stamp']-df['stamp'].shift(1), 1)).value_counts())

df['stamp'] = df['stamp'].apply(lambda x: lmd_stamp(x))

display(df['stamp'].iloc[0])

# df.to_csv(f"C:\\Users\westwell\Documents\.WorkDocuments\github\wwl_ds_projects\data\processed\{name}.csv")

c:\Users\westwell\Documents\.WorkDocuments\github\wwl_ds_projects\localization_module\data\raw\localization2_optimized_pose_2024-05-18_073904.csv
c:\Users\westwell\Documents\.WorkDocuments\github\wwl_ds_projects\localization_module\data\raw\localization2_optimized_pose_2024-05-20_052736.csv


Index(['stamp', 'optimized_pose_x', 'optimized_pose_y', 'optimized_pose_yaw'], dtype='object')

1716026889.599762

stamp
0.2       7926
0.1        799
0.4        107
0.6         26
0.3         26
          ... 
1225.8       1
1.4          1
12.9         1
1764.2       1
15.5         1
Name: count, Length: 78, dtype: int64

1716026889.5

In [6]:
def get_info(folder_path):
    path_list, df_list = [], []
    num = 0

    for item in os.listdir(folder_path):
        if item.lower().endswith('.csv'):
            path_list.append(os.path.join(folder_path, item))

    for num in range(0,len(path_list)):
        p = path_list[num]
        df = pd.read_csv(p, index_col=0)
        print(num,
                os.path.basename(p), 
                # df.columns, 
                df.shape)

        if num == 0: 
            col = ['stamp', 'container_pose_x', 'container_pose_y','container_updated_pose_x', 'container_updated_pose_y',]
        elif num == 1: col = ['stamp', 'correct_pose_by_wall_x', 'correct_pose_by_wall_y',]
        elif num == 2: col = ['stamp', 'gnss_x', 'gnss_y', 'gnss_fsm_status']
        elif num == 3: col = ['stamp', 'base_footprint_x', 'base_footprint_y',]
        elif num == 4: col = ['stamp', 'optimized_pose_x', 'optimized_pose_y',]
        elif num == 5: col = ['stamp', 'qc_match_pose_x', 'qc_match_pose_y','qc_match_updated_pose_x', 'qc_match_updated_pose_y']
        df = df[col].copy()

        df_list.append(df)
    return df_list
    
    
folder_path = os.path.abspath(os.path.join('data','processed'))
df_list = get_info(folder_path=folder_path)

0 localization2_container_pose.csv (137066, 17)
1 localization2_correct_pose_by_wall.csv (325756, 7)
2 localization2_gnss_pose.csv (131155, 16)
3 localization2_localization_odom.csv (88538, 34)
4 localization2_optimized_pose.csv (9037, 4)
5 localization2_qc_match_pose.csv (63586, 17)


In [7]:
gps1 = pd.merge(df_list[2], df_list[0], how='left',on='stamp')
gps2 = pd.merge(gps1, df_list[1],how='left',on='stamp' )
gps3 = pd.merge(gps2, df_list[3],how='left',on='stamp' )
gps4 = pd.merge(gps3, df_list[4],how='left',on='stamp' )
gps5 = pd.merge(gps4, df_list[5],how='left',on='stamp' )

print(gps5.shape)

(131882, 18)


In [10]:
gps5.head()
display(gps5.columns)
export_p1 = os.path.abspath(os.path.join('data', 'export'))
print(export_p1)
export_p1 = 'C:\\Users\westwell\Documents\\.WorkDocuments\github\wwl_ds_projects\localization_module\data\export'
gps5.to_csv(export_p1)

Index(['stamp', 'gnss_x', 'gnss_y', 'gnss_fsm_status', 'container_pose_x',
       'container_pose_y', 'container_updated_pose_x',
       'container_updated_pose_y', 'correct_pose_by_wall_x',
       'correct_pose_by_wall_y', 'base_footprint_x', 'base_footprint_y',
       'optimized_pose_x', 'optimized_pose_y', 'qc_match_pose_x',
       'qc_match_pose_y', 'qc_match_updated_pose_x',
       'qc_match_updated_pose_y'],
      dtype='object')

c:\Users\westwell\Documents\.WorkDocuments\github\wwl_ds_projects\localization_module\data\export


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\westwell\\Documents\\.WorkDocuments\\github\\wwl_ds_projects\\localization_module\\data\\export'

In [9]:
def melt_data(df):
    # Identify columns that end with _x and _y
    x_cols = [col for col in gps5.columns if col.endswith('_x')]
    y_cols = [col for col in gps5.columns if col.endswith('_y')]

    # Create a list of tuples (x_col, y_col)
    xy_cols = list(zip(x_cols, y_cols))

    # Initialize the result DataFrame
    result = pd.DataFrame()

    for x_col, y_col in xy_cols:
        src = x_col.rsplit('_', 1)[0]  # Extract the source by removing the last '_x'
        temp_df = gps5[['stamp', x_col, y_col, 'gnss_fsm_status']].copy()
        temp_df.columns = ['stamp', 'x', 'y', 'gnss_fsm_status']
        temp_df['src'] = src
        result = pd.concat([result, temp_df], ignore_index=True)

    # Reorder the columns as desired
    result = result[['stamp', 'x', 'y', 'src', 'gnss_fsm_status']]

    return result

res = melt_data(gps5)

print(res)

res.to_csv("C:\\Users\westwell\Documents\\.WorkDocuments\github\\wwl_ds_projects\data\\processed\\results.csv")


                stamp            x           y                    src  \
0        1.716181e+09  1001.401664  565.097769                   gnss   
1        1.716181e+09  1001.401511  565.097814                   gnss   
2        1.716181e+09  1001.401618  565.097958                   gnss   
3        1.716181e+09  1001.401760  565.098114                   gnss   
4        1.716181e+09  1001.401928  565.098294                   gnss   
...               ...          ...         ...                    ...   
1055051  1.716208e+09          NaN         NaN  qc_match_updated_pose   
1055052  1.716208e+09          NaN         NaN  qc_match_updated_pose   
1055053  1.716208e+09          NaN         NaN  qc_match_updated_pose   
1055054  1.716208e+09          NaN         NaN  qc_match_updated_pose   
1055055  1.716208e+09          NaN         NaN  qc_match_updated_pose   

         gnss_fsm_status  
0                    1.0  
1                    1.0  
2                    1.0  
3              

OSError: Cannot save file into a non-existent directory: 'C:\Users\westwell\Documents\.WorkDocuments\github\wwl_ds_projects\data\processed'

In [176]:
res.dtypes

stamp              float64
x                  float64
y                  float64
src                 object
gnss_fsm_status    float64
dtype: object

In [ ]:
# gnss -< gps -> 0,2,4,6,8 # gnss status
# container pose
# correct pose by wall
# base footprint 
# optimized
# qc match pose

# Feature Eng

### Location Section

In [203]:
with open("C:\\Users\westwell\Documents\\.WorkDocuments\github\wwl_ds_projects\data\cfg\TS_map.json", 'r') as f:
    map_config = json.load(f)['YARD']
print(map_config)

def lmd_section_tag(x, y):
    if map_config['x_min'] <= x <= map_config['x_max'] and map_config['y_min'] <= y <= map_config['y_max']:
        return 'YARD'
    else: return 'QC'

res['YARD_or_QC'] = res.apply(lambda x: lmd_section_tag(x['x'], x['y']), axis=1)

# res.to_csv("C:\\Users\westwell\Documents\\.WorkDocuments\github\\wwl_ds_projects\data\\processed\\results.csv")


{'x_min': 880, 'x_max': 1864, 'y_min': 243, 'y_max': 530}


### Euclidean Distance

In [204]:
df = res[res['gnss_fsm_status']==0].copy()
print(res.shape, df.shape)

(1055056, 6) (447344, 6)


In [205]:
gps5[gps5['gnss_fsm_status']==0]['base_footprint_y'].isna().value_counts()

base_footprint_y
True     30797
False    25121
Name: count, dtype: int64